# Install Required Segmentation Packages

In [ ]:
!pip install -q segmentation-models-pytorch

In [ ]:
import sys
sys.path.append("/kaggle/input/smp-lib-zip/") 

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import cv2
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp
import warnings
import time
from collections import defaultdict
import copy
import random

In [ ]:
BASE_PATH = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
TRAIN_AUTHENTIC_PATH = os.path.join(BASE_PATH, "train_images/authentic")
TRAIN_FORGED_PATH = os.path.join(BASE_PATH, "train_images/forged")
TRAIN_MASKS_PATH = os.path.join(BASE_PATH, "train_masks")
TEST_IMAGES_PATH = os.path.join(BASE_PATH, "test_images")


print("="*60)
print(" DATA LOADING AND BASIC STATISTICS")
print("="*60)

authentic_images = sorted([f for f in os.listdir(TRAIN_AUTHENTIC_PATH) if f.endswith('.png')])
forged_images = sorted([f for f in os.listdir(TRAIN_FORGED_PATH) if f.endswith('.png')])
mask_files = sorted([f for f in os.listdir(TRAIN_MASKS_PATH) if f.endswith('.npy')])
test_images = sorted([f for f in os.listdir(TEST_IMAGES_PATH) if f.endswith('.png')])

print(f"Number of AUTHENTIC images: {len(authentic_images)}")
print(f"Number of FORGED images: {len(forged_images)}")
print(f"Number of MASK files: {len(mask_files)}")
print(f"Number of TEST images: {len(test_images)}")
print(f"\nTotal TRAIN images: {len(authentic_images) + len(forged_images)}")
print(f"Forged/Authentic ratio: {len(forged_images)/len(authentic_images):.2f}")
print("\n")

# Data Visualization

In [ ]:
def visualize_forgery_overlay(forged_list, masks_path, n_samples=4): 
    selected_images = random.sample(forged_list, min(n_samples, len(forged_list)))
    
    fig, axes = plt.subplots(len(selected_images), 3, figsize=(18, 6*len(selected_images)))
    fig.suptitle('Forged Images with Mask Overlay', fontsize=16, fontweight='bold')
    
    for i, forged_file in enumerate(selected_images):
        img_path = os.path.join(TRAIN_FORGED_PATH, forged_file)
        img = np.array(Image.open(img_path))
        mask_name = forged_file.replace('.png', '.npy')
        mask_path = os.path.join(masks_path, mask_name)
        
        if os.path.exists(mask_path):
            mask = np.load(mask_path)
            if len(mask.shape) == 3:
                combined_mask = np.max(mask, axis=0)
                n_regions = mask.shape[0]
            else:
                combined_mask = mask
                n_regions = 1
            
            axes[i, 0].imshow(img)
            axes[i, 0].set_title(f'Original Image\n{forged_file}', fontsize=10)
            axes[i, 0].axis('off')

            axes[i, 1].imshow(combined_mask, cmap='hot')
            axes[i, 1].set_title(f'Forgery Mask\n{n_regions} region(s)', fontsize=10)
            axes[i, 1].axis('off')
            
            overlay = img.copy()
            if len(img.shape) == 2:  
                overlay = cv2.cvtColor(overlay, cv2.COLOR_GRAY2RGB)
            
            red_mask = np.zeros_like(overlay)
            red_mask[:,:,0] = combined_mask * 255  
            alpha = 0.4
            blended = cv2.addWeighted(overlay, 1-alpha, red_mask, alpha, 0)
            
            axes[i, 2].imshow(blended)
            axes[i, 2].set_title('Overlay (Red = Forged)', fontsize=10)
            axes[i, 2].axis('off')
        else:
            for j in range(3):
                axes[i, j].text(0.5, 0.5, 'MASK NOT FOUND', 
                               ha='center', va='center', fontsize=12)
                axes[i, j].axis('off')
    
    plt.tight_layout()
    plt.show()

visualize_forgery_overlay(forged_images, TRAIN_MASKS_PATH, n_samples=8)

# Data Preparation

In [ ]:
image_names = []
image_paths = []
mask_paths = []
labels = []  
categories = []

for img_name in authentic_images:
    image_names.append(img_name)
    image_paths.append(os.path.join(TRAIN_AUTHENTIC_PATH, img_name))
    mask_paths.append(None)  
    labels.append(0)
    categories.append('authentic')


for img_name in forged_images:
    image_names.append(img_name)
    image_paths.append(os.path.join(TRAIN_FORGED_PATH, img_name))
    
    mask_name = img_name.replace('.png', '.npy')
    mask_path = os.path.join(TRAIN_MASKS_PATH, mask_name)
    mask_paths.append(mask_path if os.path.exists(mask_path) else None)
    
    labels.append(1)
    categories.append('forged')

train_df = pd.DataFrame({
    'image_name': image_names,
    'image_path': image_paths,
    'mask_path': mask_paths,
    'label': labels,
    'category': categories
})

print(f"\nTraining DataFrame created!")
print(f"Total samples: {len(train_df)}")
print(f"\nClass distribution:")
print(train_df['category'].value_counts())
print(f"\nFirst few rows:")
print(train_df.head())

forged_df = train_df[train_df['category'] == 'forged']
missing_masks = forged_df['mask_path'].isna().sum()
print(f"\nForged images without masks: {missing_masks}")

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(
    train_df, 
    test_size=0.2, 
    random_state=42,
    stratify=train_df['label']
)

train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)

print(f"\nTrain set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")

print(f"\nTrain set distribution:")
print(train_data['category'].value_counts())
print(f"  Forged percentage: {(train_data['label'].sum() / len(train_data)) * 100:.2f}%")

print(f"\nValidation set distribution:")
print(val_data['category'].value_counts())
print(f"  Forged percentage: {(val_data['label'].sum() / len(val_data)) * 100:.2f}%")

# Data Loader Helper Function

In [ ]:
def load_image(image_path, target_size=(512, 512)):
    img = Image.open(image_path).convert('RGB')
    img = img.resize(target_size, Image.BILINEAR)
    img_array = np.array(img, dtype=np.float32) / 255.0  # Normalize to [0, 1]
    return img_array

def load_mask(mask_path, target_size=(512, 512)):
    if mask_path is None or not os.path.exists(mask_path):
        
        return np.zeros(target_size[::-1], dtype=np.float32)
    
   
    mask = np.load(mask_path)
    

    if len(mask.shape) == 3:
        mask = np.max(mask, axis=0)
    
    mask_resized = cv2.resize(mask.astype(np.float32), target_size, interpolation=cv2.INTER_NEAREST)
    mask_binary = (mask_resized > 0.5).astype(np.float32)
    
    return mask_binary

def load_sample(df, idx, target_size=(512, 512)):
    row = df.iloc[idx]
    
    image = load_image(row['image_path'], target_size)
    mask = load_mask(row['mask_path'], target_size)
    label = row['label']
    
    return image, mask, label

In [ ]:
TARGET_SIZE = (512, 512)

print(f"\nTesting data loading with target size: {TARGET_SIZE}")
print("Loading 3 samples from training set...\n")

forged_indices = train_data[train_data['category'] == 'forged'].index.tolist()
random_indices = random.sample(forged_indices, 5)

fig, axes = plt.subplots(5, 3, figsize=(15, 12))

for i, forged_idx in enumerate(random_indices):
    image, mask, label = load_sample(train_data, forged_idx, TARGET_SIZE)
    
    # Original image
    axes[i, 0].imshow(image)
    axes[i, 0].set_title(f'Image (Label: {label})', fontsize=10)
    axes[i, 0].axis('off')
    
    # Mask
    axes[i, 1].imshow(mask, cmap='hot')
    axes[i, 1].set_title('Mask', fontsize=10)
    axes[i, 1].axis('off')
    
    # Overlay
    overlay = (image * 255).astype(np.uint8)
    red_overlay = np.zeros_like(overlay)
    red_overlay[:, :, 0] = (mask * 255).astype(np.uint8)
    blended = cv2.addWeighted(overlay, 0.6, red_overlay, 0.4, 0)
    
    axes[i, 2].imshow(blended)
    axes[i, 2].set_title('Overlay', fontsize=10)
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

print("\nData loading test completed successfully!")
print(f"Image shape: {image.shape}")
print(f"Mask shape: {mask.shape}")
print(f"Image value range: [{image.min():.3f}, {image.max():.3f}]")
print(f"Mask unique values: {np.unique(mask)}")

# Data Augumentation

In [ ]:
print(f"\nPyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
print("\n")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}\n")


print("="*50)
print("CREATING AUGMENTATION PIPELINES")
print("="*50)

train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30, p=0.5, border_mode=cv2.BORDER_CONSTANT, value=0),
    A.ShiftScaleRotate(
        shift_limit=0.1,
        scale_limit=0.15,
        rotate_limit=15,
        border_mode=cv2.BORDER_CONSTANT,
        value=0,
        p=0.5
    ),
    
    A.OneOf([
        A.OpticalDistortion(distort_limit=0.1, p=1.0),
        A.GridDistortion(num_steps=5, distort_limit=0.1, p=1.0),
        A.ElasticTransform(alpha=1, sigma=50, p=1.0),
    ], p=0.3),
    
    A.OneOf([
        A.RandomBrightnessContrast(
            brightness_limit=0.2,
            contrast_limit=0.2,
            p=1.0
        ),
        A.HueSaturationValue(
            hue_shift_limit=10,
            sat_shift_limit=20,
            val_shift_limit=20,
            p=1.0
        ),
        A.RandomGamma(gamma_limit=(80, 120), p=1.0),
    ], p=0.5),
    
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
        A.GaussianBlur(blur_limit=(3, 5), p=1.0),
        A.MotionBlur(blur_limit=5, p=1.0),
    ], p=0.3),
    
    A.CLAHE(clip_limit=2.0, p=0.3),
    A.RandomShadow(
        shadow_roi=(0, 0.5, 1, 1),
        num_shadows_lower=1,
        num_shadows_upper=2,
        shadow_dimension=5,
        p=0.2
    ),
    
    A.Normalize(
        mean=[0.485, 0.456, 0.406],  
        std=[0.229, 0.224, 0.225],
        max_pixel_value=1.0,
    ),
    ToTensorV2(),
])

val_transform = A.Compose([
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
        max_pixel_value=1.0,
    ),
    ToTensorV2(),
])

# PyTorch Dataset Generator

In [ ]:
class ForgeryDetectionDataset(Dataset):
    
    def __init__(self, dataframe, transform=None, target_size=(512, 512)):
    
        self.df = dataframe.reset_index(drop=True)
        self.transform = transform
        self.target_size = target_size
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = row['image_path']
        mask_path = row['mask_path']
        label = row['label']
        
        image = Image.open(image_path).convert('RGB')
        image = image.resize(self.target_size, Image.BILINEAR)
        image = np.array(image, dtype=np.float32) / 255.0
        
    
        if mask_path is not None and os.path.exists(mask_path):
            mask = np.load(mask_path)
            
            if len(mask.shape) == 3:
                mask = np.max(mask, axis=0)
            
            mask = cv2.resize(
                mask.astype(np.float32),
                self.target_size,
                interpolation=cv2.INTER_NEAREST
            )
            
        
            mask = (mask > 0.5).astype(np.float32)
        else:
        
            mask = np.zeros(self.target_size, dtype=np.float32)
        
      
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']
        else:
           
            image = torch.from_numpy(image).permute(2, 0, 1) 
            mask = torch.from_numpy(mask).unsqueeze(0) 
        
        if len(mask.shape) == 2:
            mask = mask.unsqueeze(0)
        
        return {
            'image': image,
            'mask': mask,
            'label': torch.tensor(label, dtype=torch.float32)
        }

In [ ]:
BATCH_SIZE = 6
NUM_WORKERS = 2
TARGET_SIZE = (512, 512)

train_dataset = ForgeryDetectionDataset(
    dataframe=train_data,
    transform=train_transform,
    target_size=TARGET_SIZE
)

val_dataset = ForgeryDetectionDataset(
    dataframe=val_data,
    transform=val_transform,
    target_size=TARGET_SIZE
)

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True if torch.cuda.is_available() else False,
    drop_last=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"✓ Dataloaders created")
print(f"  - Batch size: {BATCH_SIZE}")
print(f"  - Training batches: {len(train_loader)}")
print(f"  - Validation batches: {len(val_loader)}")
print(f"  - Training samples: {len(train_dataset)}")
print(f"  - Validation samples: {len(val_dataset)}")
print("\n")

# Validate DataLoader

In [ ]:
sample_batch = next(iter(train_loader))

print(f"Sample batch shapes:")
print(f"  - Images: {sample_batch['image'].shape}")
print(f"  - Masks: {sample_batch['mask'].shape}")
print(f"  - Labels: {sample_batch['label'].shape}")

print(f"\nImage tensor stats:")
print(f"  - Min: {sample_batch['image'].min():.3f}")
print(f"  - Max: {sample_batch['image'].max():.3f}")
print(f"  - Mean: {sample_batch['image'].mean():.3f}")

print(f"\nMask tensor stats:")
print(f"  - Unique values: {torch.unique(sample_batch['mask'])}")
print(f"  - Forged pixels ratio: {sample_batch['mask'].mean():.4f}")


def denormalize(image):
    mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
    std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    image = image * std + mean
    return torch.clamp(image, 0, 1)

fig, axes = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle('Augmented Training Samples', fontsize=16, fontweight='bold')

for i in range(3):
    # Get sample
    img = denormalize(sample_batch['image'][i]).permute(1, 2, 0).cpu().numpy()
    mask = sample_batch['mask'][i, 0].cpu().numpy()
    label = sample_batch['label'][i].item()
    
    # Plot
    axes[i, 0].imshow(img)
    axes[i, 0].set_title(f'Augmented Image (Label: {label:.0f})')
    axes[i, 0].axis('off')
    
    axes[i, 1].imshow(mask, cmap='hot')
    axes[i, 1].set_title('Augmented Mask')
    axes[i, 1].axis('off')
    
    # Overlay
    overlay = (img * 255).astype(np.uint8)
    red_overlay = np.zeros_like(overlay)
    red_overlay[:,:,0] = (mask * 255).astype(np.uint8)
    blended = cv2.addWeighted(overlay, 0.6, red_overlay, 0.4, 0)
    
    axes[i, 2].imshow(blended)
    axes[i, 2].set_title('Overlay')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

# Model Aricturaccture

In [ ]:
class SegformerForgeryDetector(nn.Module):
    def __init__(
        self,
        encoder_name='mit_b2',  # MiT Transformer backbone
        encoder_weights='imagenet',  # pretrained weights
        in_channels=3,
        classes=1,
        activation=None
    ):
        super(SegformerForgeryDetector, self).__init__()
        
        self.model = smp.Segformer(
            encoder_name=encoder_name,
            encoder_weights=encoder_weights,
            in_channels=in_channels,
            classes=classes,
            activation=activation
        )

    def forward(self, x):
        return self.model(x)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SegformerForgeryDetector(
    encoder_name='mit_b2',
    encoder_weights='imagenet',
    in_channels=3,
    classes=1,
    activation=None
).to(device)

model = model.to(device)

print(f"✓ SegFormer model created")
print(f"  - Encoder: SegFormer ")
print(f"  - Pre-trained weights: MiT-B2")
print(f"  - Input channels: 3 (RGB)")
print(f"  - Output classes: 1 (binary segmentation)")
print(f"  - Device: {device}")
print("\n")


def count_parameters(model):
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable

total_params, trainable_params = count_parameters(model)

print(f"Model Parameters:")
print(f"  - Total: {total_params:,}")
print(f"  - Trainable: {trainable_params:,}")
print(f"  - Size: ~{total_params * 4 / (1024**2):.2f} MB (FP32)")


print("\nTesting forward pass...")
with torch.no_grad():
    dummy_input = torch.randn(2, 3, 512, 512).to(device)
    dummy_output = model(dummy_input)
    print(f"  - Input shape: {dummy_input.shape}")
    print(f"  - Output shape: {dummy_output.shape}")
    print("✓ Forward pass successful!")
print("\n")



print("="*60)
print(" DEFINING LOSS FUNCTIONS")
print("="*60)

class DiceLoss(nn.Module):  
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth
    
    def forward(self, predictions, targets):
        predictions = torch.sigmoid(predictions)
        
        predictions = predictions.view(-1)
        targets = targets.view(-1)
        
        intersection = (predictions * targets).sum()
        dice = (2. * intersection + self.smooth) / (
            predictions.sum() + targets.sum() + self.smooth
        )
        
        return 1 - dice

class CombinedLoss(nn.Module):
    
    def __init__(self, bce_weight=0.5, dice_weight=0.5):
        super(CombinedLoss, self).__init__()
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight
        self.bce = nn.BCEWithLogitsLoss()
        self.dice = DiceLoss()
    
    def forward(self, predictions, targets):
        bce_loss = self.bce(predictions, targets)
        dice_loss = self.dice(predictions, targets)
        
        combined = self.bce_weight * bce_loss + self.dice_weight * dice_loss
        
        return combined, bce_loss, dice_loss

criterion = CombinedLoss(bce_weight=0.5, dice_weight=0.5)

print("✓ Loss functions created")
print("  - BCE Loss: Binary Cross-Entropy with Logits")
print("  - Dice Loss: Custom implementation")
print("  - Combined Loss: 0.5 * BCE + 0.5 * Dice")
print("\n")


print("="*60)
print(" CONFIGURING OPTIMIZER AND SCHEDULER")
print("="*60)

LEARNING_RATE = 3e-4
WEIGHT_DECAY = 1e-5

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY
)


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=5,
    verbose=True,
    min_lr=1e-7
)

print(f"✓ Optimizer: AdamW")
print(f"  - Learning rate: {LEARNING_RATE}")
print(f"  - Weight decay: {WEIGHT_DECAY}")

print(f"\n✓ Scheduler: ReduceLROnPlateau")
print(f"  - Mode: min (reduce on plateau)")
print(f"  - Factor: 0.5 (halve LR)")
print(f"  - Patience: 5 epochs")
print(f"  - Min LR: 1e-7")
print("\n")

print("="*60)
print("DEFINING EVALUATION METRICS")
print("="*60)

def dice_coefficient(predictions, targets, threshold=0.5, smooth=1.0):

    predictions = (torch.sigmoid(predictions) > threshold).float()
    predictions = predictions.view(-1)
    targets = targets.view(-1)
    
    intersection = (predictions * targets).sum()
    dice = (2. * intersection + smooth) / (
        predictions.sum() + targets.sum() + smooth
    )
    
    return dice.item()

def iou_score(predictions, targets, threshold=0.5, smooth=1.0):
    predictions = (torch.sigmoid(predictions) > threshold).float()
    predictions = predictions.view(-1)
    targets = targets.view(-1)
    
    intersection = (predictions * targets).sum()
    union = predictions.sum() + targets.sum() - intersection
    
    iou = (intersection + smooth) / (union + smooth)
    
    return iou.item()

def pixel_accuracy(predictions, targets, threshold=0.5):
   
    predictions = (torch.sigmoid(predictions) > threshold).float()
    correct = (predictions == targets).float().sum()
    total = targets.numel()
    
    return (correct / total).item()

print("✓ Evaluation metrics defined")
print("  - Dice Coefficient")
print("  - IoU (Intersection over Union)")
print("  - Pixel Accuracy")
print("\n")

# Configuration

In [ ]:
CONFIG = {
    'epochs': 10,
    'early_stopping_patience': 7,
    'best_model_metric': 'val_dice',  
    'checkpoint_dir': 'checkpoints',
    'save_best_only': True,
}

print(f"""
Training Configuration:
  - Epochs: {CONFIG['epochs']}
  - Early stopping patience: {CONFIG['early_stopping_patience']}
  - Best model metric: {CONFIG['best_model_metric']}
  - Checkpoint directory: {CONFIG['checkpoint_dir']}
  - Save best only: {CONFIG['save_best_only']}
""")

os.makedirs(CONFIG['checkpoint_dir'], exist_ok=True)


class AverageMeter:

    def __init__(self):
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

class EarlyStopping:
    def __init__(self, patience=7, mode='min', delta=0.0001):
        self.patience = patience
        self.mode = mode
        self.delta = delta
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        
    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
            return False
        
        if self.mode == 'min':
            improved = score < (self.best_score - self.delta)
        else:
            improved = score > (self.best_score + self.delta)
        
        if improved:
            self.best_score = score
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        
        return self.early_stop

print("✓ Utility classes created:")
print("  - AverageMeter: Track running averages")
print("  - EarlyStopping: Stop training early if no improvement")
print("\n")


def train_one_epoch(model, dataloader, criterion, optimizer, device, epoch):
    model.train()
    
    loss_meter = AverageMeter()
    bce_meter = AverageMeter()
    dice_loss_meter = AverageMeter()
    dice_score_meter = AverageMeter()
    iou_meter = AverageMeter()
    
    pbar = tqdm(dataloader, desc=f'Epoch {epoch} [TRAIN]')
    
    for batch_idx, batch in enumerate(pbar):
        
        images = batch['image'].to(device)
        masks = batch['mask'].to(device)
        
        outputs = model(images)
        
        loss, bce_loss, dice_loss = criterion(outputs, masks)
        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            dice_score = dice_coefficient(outputs, masks)
            iou = iou_score(outputs, masks)
        
        batch_size = images.size(0)
        loss_meter.update(loss.item(), batch_size)
        bce_meter.update(bce_loss.item(), batch_size)
        dice_loss_meter.update(dice_loss.item(), batch_size)
        dice_score_meter.update(dice_score, batch_size)
        iou_meter.update(iou, batch_size)
        
        pbar.set_postfix({
            'loss': f'{loss_meter.avg:.4f}',
            'dice': f'{dice_score_meter.avg:.4f}',
            'iou': f'{iou_meter.avg:.4f}'
        })
    
    return {
        'loss': loss_meter.avg,
        'bce_loss': bce_meter.avg,
        'dice_loss': dice_loss_meter.avg,
        'dice_score': dice_score_meter.avg,
        'iou': iou_meter.avg
    }

def validate(model, dataloader, criterion, device, epoch):
    model.eval()
    
    loss_meter = AverageMeter()
    bce_meter = AverageMeter()
    dice_loss_meter = AverageMeter()
    dice_score_meter = AverageMeter()
    iou_meter = AverageMeter()
    
    pbar = tqdm(dataloader, desc=f'Epoch {epoch} [VALID]')
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(pbar):
            
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)
            
            outputs = model(images)
            
            loss, bce_loss, dice_loss = criterion(outputs, masks)
            
            dice_score = dice_coefficient(outputs, masks)
            iou = iou_score(outputs, masks)
            
            batch_size = images.size(0)
            loss_meter.update(loss.item(), batch_size)
            bce_meter.update(bce_loss.item(), batch_size)
            dice_loss_meter.update(dice_loss.item(), batch_size)
            dice_score_meter.update(dice_score, batch_size)
            iou_meter.update(iou, batch_size)
            
            pbar.set_postfix({
                'loss': f'{loss_meter.avg:.4f}',
                'dice': f'{dice_score_meter.avg:.4f}',
                'iou': f'{iou_meter.avg:.4f}'
            })
    
    return {
        'loss': loss_meter.avg,
        'bce_loss': bce_meter.avg,
        'dice_loss': dice_loss_meter.avg,
        'dice_score': dice_score_meter.avg,
        'iou': iou_meter.avg
    }

# Model Training Pipeline

In [ ]:
history = defaultdict(list)
best_dice = 0.0
best_model_wts = copy.deepcopy(model.state_dict())

early_stopping = EarlyStopping(
    patience=CONFIG['early_stopping_patience'],
    mode='max',  
    delta=0.001
)

training_start_time = time.time()

print(f"Starting training for {CONFIG['epochs']} epochs...")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Device: {device}")
print("="*80)
print("\n")


for epoch in range(1, CONFIG['epochs'] + 1):
    epoch_start_time = time.time()
    
    print(f"\n{'='*80}")
    print(f"EPOCH {epoch}/{CONFIG['epochs']}")
    print(f"{'='*80}")
    
    train_metrics = train_one_epoch(
        model=model,
        dataloader=train_loader,
        criterion=criterion,
        optimizer=optimizer,
        device=device,
        epoch=epoch
    )
    
    val_metrics = validate(
        model=model,
        dataloader=val_loader,
        criterion=criterion,
        device=device,
        epoch=epoch
    )
    
    scheduler.step(val_metrics['loss'])
    
    current_lr = optimizer.param_groups[0]['lr']
    
    epoch_time = time.time() - epoch_start_time
    
    print(f"\n{'='*80}")
    print(f"EPOCH {epoch} SUMMARY")
    print(f"{'='*80}")
    print(f"Time: {epoch_time:.2f}s | LR: {current_lr:.2e}")
    print(f"\nTrain Metrics:")
    print(f"  Loss: {train_metrics['loss']:.4f} | BCE: {train_metrics['bce_loss']:.4f} | Dice Loss: {train_metrics['dice_loss']:.4f}")
    print(f"  Dice Score: {train_metrics['dice_score']:.4f} | IoU: {train_metrics['iou']:.4f}")
    print(f"\nValidation Metrics:")
    print(f"  Loss: {val_metrics['loss']:.4f} | BCE: {val_metrics['bce_loss']:.4f} | Dice Loss: {val_metrics['dice_loss']:.4f}")
    print(f"  Dice Score: {val_metrics['dice_score']:.4f} | IoU: {val_metrics['iou']:.4f}")
    
    history['epoch'].append(epoch)
    history['lr'].append(current_lr)
    history['train_loss'].append(train_metrics['loss'])
    history['train_dice'].append(train_metrics['dice_score'])
    history['train_iou'].append(train_metrics['iou'])
    history['val_loss'].append(val_metrics['loss'])
    history['val_dice'].append(val_metrics['dice_score'])
    history['val_iou'].append(val_metrics['iou'])
    
    if val_metrics['dice_score'] > best_dice:
        best_dice = val_metrics['dice_score']
        best_model_wts = copy.deepcopy(model.state_dict())
        
        checkpoint_path = os.path.join(
            CONFIG['checkpoint_dir'],
            f'best_model_epoch{epoch}_dice{best_dice:.4f}.pth'
        )
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'best_dice': best_dice,
            'val_metrics': val_metrics,
        }, checkpoint_path)
        
        print(f"\n🎯 New best model saved! Dice: {best_dice:.4f}")
        print(f"   Saved to: {checkpoint_path}")
    
    if early_stopping(val_metrics['dice_score']):
        print(f"\n  Early stopping triggered after {epoch} epochs")
        print(f"   No improvement for {CONFIG['early_stopping_patience']} epochs")
        break
    
    print(f"{'='*80}\n")



training_time = time.time() - training_start_time

print("\n")
print("="*80)
print("TRAINING COMPLETED!")
print("="*80)
print(f"Total training time: {training_time/60:.2f} minutes")
print(f"Best validation Dice score: {best_dice:.4f}")
print(f"Total epochs trained: {len(history['epoch'])}")
print("="*80)
print("\n")


model.load_state_dict(best_model_wts)
print("✓ Best model weights loaded into model")
print("\n")


history_df = pd.DataFrame(history)

fig, axes = plt.subplots(2, 3, figsize=(20, 12))
fig.suptitle('Training History', fontsize=16, fontweight='bold')

axes[0, 0].plot(history_df['epoch'], history_df['train_loss'], 
                label='Train Loss', marker='o', linewidth=2)
axes[0, 0].plot(history_df['epoch'], history_df['val_loss'], 
                label='Val Loss', marker='s', linewidth=2)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Loss Curve')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(history_df['epoch'], history_df['train_dice'], 
                label='Train Dice', marker='o', linewidth=2, color='green')
axes[0, 1].plot(history_df['epoch'], history_df['val_dice'], 
                label='Val Dice', marker='s', linewidth=2, color='orange')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Dice Score')
axes[0, 1].set_title('Dice Score Curve')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[0, 2].plot(history_df['epoch'], history_df['train_iou'], 
                label='Train IoU', marker='o', linewidth=2, color='purple')
axes[0, 2].plot(history_df['epoch'], history_df['val_iou'], 
                label='Val IoU', marker='s', linewidth=2, color='brown')
axes[0, 2].set_xlabel('Epoch')
axes[0, 2].set_ylabel('IoU Score')
axes[0, 2].set_title('IoU Score Curve')
axes[0, 2].legend()
axes[0, 2].grid(True, alpha=0.3)

axes[1, 0].plot(history_df['epoch'], history_df['lr'], 
                marker='o', linewidth=2, color='red')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Learning Rate')
axes[1, 0].set_title('Learning Rate Schedule')
axes[1, 0].set_yscale('log')
axes[1, 0].grid(True, alpha=0.3)


axes[1, 1].scatter(history_df['train_loss'], history_df['val_loss'], 
                   c=history_df['epoch'], cmap='viridis', s=100, alpha=0.6)
axes[1, 1].plot([history_df['train_loss'].min(), history_df['train_loss'].max()],
                [history_df['train_loss'].min(), history_df['train_loss'].max()],
                'r--', linewidth=2, label='Perfect fit')
axes[1, 1].set_xlabel('Train Loss')
axes[1, 1].set_ylabel('Val Loss')
axes[1, 1].set_title('Train vs Val Loss')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

axes[1, 2].scatter(history_df['train_dice'], history_df['val_dice'], 
                   c=history_df['epoch'], cmap='viridis', s=100, alpha=0.6)
axes[1, 2].plot([history_df['train_dice'].min(), history_df['train_dice'].max()],
                [history_df['train_dice'].min(), history_df['train_dice'].max()],
                'r--', linewidth=2, label='Perfect fit')
axes[1, 2].set_xlabel('Train Dice')
axes[1, 2].set_ylabel('Val Dice')
axes[1, 2].set_title('Train vs Val Dice')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✓ Training curves plotted")
print("\n")



best_epoch_idx = history_df['val_dice'].idxmax()
best_epoch_data = history_df.iloc[best_epoch_idx]

print(f"\nBest Epoch: {int(best_epoch_data['epoch'])}")
print(f"\nMetrics at Best Epoch:")
print(f"  Train Loss: {best_epoch_data['train_loss']:.4f}")
print(f"  Val Loss: {best_epoch_data['val_loss']:.4f}")
print(f"  Train Dice: {best_epoch_data['train_dice']:.4f}")
print(f"  Val Dice: {best_epoch_data['val_dice']:.4f}")
print(f"  Train IoU: {best_epoch_data['train_iou']:.4f}")
print(f"  Val IoU: {best_epoch_data['val_iou']:.4f}")
print(f"  Learning Rate: {best_epoch_data['lr']:.2e}")

# Check for overfitting
dice_gap = best_epoch_data['train_dice'] - best_epoch_data['val_dice']
loss_gap = best_epoch_data['val_loss'] - best_epoch_data['train_loss']

print(f"\nOverfitting Analysis:")
print(f"  Dice gap (Train - Val): {dice_gap:.4f}")
print(f"  Loss gap (Val - Train): {loss_gap:.4f}")

if dice_gap > 0.1:
    print("  ⚠️  Warning: Significant overfitting detected (Dice gap > 0.1)")
elif dice_gap > 0.05:
    print("  ⚡ Moderate overfitting (Dice gap > 0.05)")
else:
    print("  ✓ Good generalization (Dice gap < 0.05)")

print("\n")

---

In [ ]:
def predict_batch(model, images, device, threshold=0.5):
    model.eval()
    with torch.no_grad():
        outputs = model(images.to(device))
        predictions = torch.sigmoid(outputs)
        binary_predictions = (predictions > threshold).float()
    return predictions, binary_predictions

def visualize_predictions(model, dataloader, device, n_samples=6, threshold=0.5):
  
    model.eval()
    
    fig, axes = plt.subplots(n_samples, 4, figsize=(20, 5*n_samples))
    fig.suptitle(f'Validation Predictions (Threshold: {threshold})', 
                 fontsize=16, fontweight='bold')
    
    batch = next(iter(dataloader))
    images = batch['image'][:n_samples].to(device)
    masks = batch['mask'][:n_samples]
    labels = batch['label'][:n_samples]
    
    with torch.no_grad():
        outputs = model(images)
        predictions = torch.sigmoid(outputs)
        binary_preds = (predictions > threshold).float()
    
    def denormalize(img):
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1).to(img.device)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1).to(img.device)
        return torch.clamp(img * std + mean, 0, 1)
    
    for i in range(n_samples):
        img = denormalize(images[i]).cpu().permute(1, 2, 0).numpy()
        axes[i, 0].imshow(img)
        axes[i, 0].set_title(f'Image (Label: {labels[i].item():.0f})')
        axes[i, 0].axis('off')
        
        gt_mask = masks[i, 0].cpu().numpy()
        axes[i, 1].imshow(gt_mask, cmap='hot', vmin=0, vmax=1)
        axes[i, 1].set_title('Ground Truth Mask')
        axes[i, 1].axis('off')
        
        pred_prob = predictions[i, 0].cpu().numpy()
        axes[i, 2].imshow(pred_prob, cmap='hot', vmin=0, vmax=1)
        axes[i, 2].set_title(f'Predicted (Prob)\nMax: {pred_prob.max():.3f}')
        axes[i, 2].axis('off')
        
        pred_binary = binary_preds[i, 0].cpu().numpy()
        axes[i, 3].imshow(pred_binary, cmap='hot', vmin=0, vmax=1)
        
        # Calculate metrics for this sample
        dice = dice_coefficient(
            outputs[i:i+1], 
            masks[i:i+1].to(device), 
            threshold=threshold
        )
        axes[i, 3].set_title(f'Binary Prediction\nDice: {dice:.3f}')
        axes[i, 3].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize predictions
print("\nVisualizing predictions on validation set...")
visualize_predictions(
    model=model,
    dataloader=val_loader,
    device=device,
    n_samples=6,
    threshold=0.5
)

print("✓ Validation predictions visualized")
print("\n")


print("="*80)
print("STEP 7.1: OPTIMIZING PREDICTION THRESHOLD")
print("="*80)

def evaluate_threshold(model, dataloader, device, threshold):
    model.eval()
    dice_scores = []
    iou_scores = []
    
    with torch.no_grad():
        for batch in dataloader:
            images = batch['image'].to(device)
            masks = batch['mask'].to(device)
            
            outputs = model(images)
            
            dice = dice_coefficient(outputs, masks, threshold=threshold)
            iou = iou_score(outputs, masks, threshold=threshold)
            
            dice_scores.append(dice)
            iou_scores.append(iou)
    
    return np.mean(dice_scores), np.mean(iou_scores)

print("Testing different thresholds...")
thresholds = [0.3,  0.7]
threshold_results = []

for thresh in tqdm(thresholds, desc="Testing thresholds"):
    dice, iou = evaluate_threshold(model, val_loader, device, thresh)
    threshold_results.append({
        'threshold': thresh,
        'dice': dice,
        'iou': iou
    })
    print(f"  Threshold {thresh:.2f}: Dice={dice:.4f}, IoU={iou:.4f}")

threshold_df = pd.DataFrame(threshold_results)
best_threshold_idx = threshold_df['dice'].idxmax()
best_threshold = threshold_df.iloc[best_threshold_idx]['threshold']
best_dice = threshold_df.iloc[best_threshold_idx]['dice']

print(f"\n✓ Best threshold: {best_threshold}")
print(f"  Dice score: {best_dice:.4f}")

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
fig.suptitle('Threshold Optimization', fontsize=16, fontweight='bold')

axes[0].plot(threshold_df['threshold'], threshold_df['dice'], 
             marker='o', linewidth=2, markersize=8, color='green')
axes[0].axvline(best_threshold, color='red', linestyle='--', 
                label=f'Best: {best_threshold}')
axes[0].set_xlabel('Threshold')
axes[0].set_ylabel('Dice Score')
axes[0].set_title('Dice Score vs Threshold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(threshold_df['threshold'], threshold_df['iou'], 
             marker='s', linewidth=2, markersize=8, color='purple')
axes[1].axvline(best_threshold, color='red', linestyle='--', 
                label=f'Best: {best_threshold}')
axes[1].set_xlabel('Threshold')
axes[1].set_ylabel('IoU Score')
axes[1].set_title('IoU Score vs Threshold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n")